In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl


In [4]:
from typing import Any, Optional


from pytorch_lightning.utilities.types import STEP_OUTPUT


class CVAE(pl.LightningModule):
    def __init__(self, input_dim, latent_dim, output_dim, lr=1e-3):
        super(CVAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.output_dim = output_dim
        self.lr = lr
        
        # Encoder layers
        self.fc1 = nn.Linear(input_dim + output_dim, 256)
        self.fc2 = nn.Linear(256, latent_dim * 2)
        
        # Decoder layers
        self.fc3 = nn.Linear(latent_dim + output_dim, 256)
        self.fc4 = nn.Linear(256, input_dim)
    
    def encode(self, x, y):
        input_combined = torch.cat((x, y), dim=1)
        hidden = F.relu(self.fc1(input_combined))
        latent_params = self.fc2(hidden)
        return latent_params
    
    def decode(self, z, y):
        latent_combined = torch.cat((z, y), dim=1)
        hidden = F.relu(self.fc3(latent_combined))
        output = self.fc4(hidden)
        return output
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z
    
    def forward(self, x, y):
        latent_params = self.encode(x, y)
        mu, logvar = latent_params[:, :self.latent_dim], latent_params[:, self.latent_dim:]
        
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z, y)
        return reconstructed, mu, logvar
    
    def loss_function(self, reconstructed, x, mu, logvar):
        reconstruction_loss = F.mse_loss(reconstructed, x, reduction='sum')
        kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return reconstruction_loss + kl_divergence
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        reconstructed, mu, logvar = self(x, y)
        loss = self.loss_function(reconstructed, x, mu, logvar)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        reconstructed, mu, logvar = self(x, y)
        loss = self.loss_function(reconstructed, x, mu, logvar)
        self.log('val_loss', loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [5]:
from pytorch_lightning.utilities import rank_zero_only
from pytorch_lightning.loggers import Logger
from pytorch_lightning.loggers.logger import rank_zero_experiment
from collections import defaultdict 


class DictLogger(Logger):

    def __init__(self):
        super().__init__()
        def def_value(): 
            return []
              
        # Defining the dict 
        self.metrics = defaultdict(def_value) 


    @property
    def name(self):
        return 'DictLogger'

    @property
    @rank_zero_experiment
    def experiment(self):
        # Return the experiment object associated with this logger.
        pass

    @property
    def version(self):
        # Return the experiment version, int or str.
        return '0.1'

    @rank_zero_only
    def log_hyperparams(self, params):
        # params is an argparse.Namespace
        # your code to record hyperparameters goes here
        pass

    @rank_zero_only
    def log_metrics(self, metrics, step):
        # metrics is a dictionary of metric names and values
        # your code to record metrics goes here
        for key in metrics.keys():
            self.metrics[key].append(metrics[key])

    @rank_zero_only
    def save(self):
        # Optional. Any code necessary to save logger data goes here
        # If you implement this, remember to call `super().save()`
        # at the start of the method (important for aggregation of metrics)
        super().save()

    @rank_zero_only
    def finalize(self, status):
        # Optional. Any code that needs to be run after training
        # finishes goes here
        pass

In [6]:

from torch.utils.data import TensorDataset

import pandas as pd
import patsy

class MyDataModule(pl.LightningDataModule):
    def __init__(self, dataset, batch_size=64):
        super().__init__()
        self.dataset = dataset
        self.batch_size = batch_size
    
    def setup(self, stage=None):
        # Split the dataset into train and validation sets
        train_size = int(0.8 * len(self.dataset))
        val_size = len(self.dataset) - train_size
        self.train_dataset, self.val_dataset = torch.utils.data.random_split(self.dataset, [train_size, val_size])
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)


Build dataset

In [7]:
df = pd.read_csv('/Users/ssilvari/Downloads/fedcombat_real_data/unified/non_harmonized_data.csv', low_memory=False, index_col=0).dropna(how='any')

formula = 'site ~ '
for i, col in enumerate(df.columns):
    if col in ['site']:
        continue
    if df[col].dtype.kind in 'biufc':
        formula += f' standardize(Q("{col}")) '
    else:
        formula += f' C(Q("{col}")) '

    if i != len(df.columns) - 1:
        formula += '+'
formula += ' -1'

Y, X = patsy.dmatrices(formula, df, return_type='matrix')
dataset = TensorDataset(torch.tensor(X).float(), torch.tensor(Y).float())

input_dim = X.shape[1]
output_dim = Y.shape[1]
latent_dim = 31

cvae = CVAE(input_dim, latent_dim, output_dim)


In [8]:
# from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.callbacks import EarlyStopping

# data_module = MyDataModule(dataset)

# logger = DictLogger()
# logger = TensorBoardLogger(save_dir='tensorboard_logs', name='MECVAE')

# early_stop_callback = EarlyStopping('val_loss')

# trainer = pl.Trainer(max_epochs=100, logger=logger, callbacks=[early_stop_callback])
# trainer.fit(cvae, data_module)

In [9]:
import optuna

def objective(trial: optuna.trial.Trial):
    latent_dim = trial.suggest_int('latent_dim', 200, 300)  # Adjust the range as per your requirements
    # latent_dim = 212
    batch_size = trial.suggest_int('batch_size', 3, 64)
    # lr = trial.suggest_loguniform('lr', 1e-5, 0.1)
    lr = 2.964865522829201e-05
    # lr = 1e-1
    
    # Create the CVAE model with the suggested latent_dim
    cvae = CVAE(input_dim, latent_dim, output_dim, lr=lr)
    
    # Create the LightningDataModule with the desired train and validation datasets
    data_module = MyDataModule(dataset, batch_size=batch_size)
    
    # Create the Trainer
    trainer = pl.Trainer(max_epochs=10)
    
    # Train the model
    trainer.fit(cvae, data_module)
    
    # Return the validation loss as the objective value for Optuna to optimize
    return trainer.callback_metrics['val_loss'].item()


Tune the best number of latent variables

In [10]:
import optuna

study = optuna.create_study(direction='minimize', storage='sqlite:///optuna.db')
study.optimize(objective, n_trials=10)  # Adjust the number of trials as per your requirements

# Print the best hyperparameters and objective value
best_trial = study.best_trial
print('Best trial - Loss: {:.4f}'.format(best_trial.value))
print('Best trial - Hyperparameters:', best_trial.params)

[I 2023-05-04 16:09:20,942] A new study created in RDB with name: no-name-a8f0af81-c0e1-4c27-8be8-b9e11ae303ea
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 151 K 
2 | fc3  | Linear | 77.8 K
3 | fc4  | Linear | 24.7 K
--------------------------------
280 K     Trainable params
0         Non-trainable params
280 K     Total params
1.123     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/ssilvari/Library/Caches/pypoetry/virtualenvs/mecvae-_b3Dgs6W-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/ssilvari/Library/Caches/pypoetry/virtualenvs/mecvae-_b3Dgs6W-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 9: 100%|██████████| 129/129 [00:02<00:00, 52.28it/s, v_num=161, train_loss=1.35e+3]

...

Epoch 9: 100%|██████████| 142/142 [00:03<00:00, 45.83it/s, v_num=170, train_loss=154.0]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 142/142 [00:03<00:00, 45.61it/s, v_num=170, train_loss=154.0]


[I 2023-05-04 16:14:14,975] Trial 9 finished with value: 2267.47509765625 and parameters: {'latent_dim': 210, 'batch_size': 41}. Best is trial 8 with value: 1513.13232421875.


Best trial - Loss: 1513.1323
Best trial - Hyperparameters: {'batch_size': 28, 'latent_dim': 226}


Results on hypertunning:

```
[I 2023-05-04 15:04:50,382] A new study created in memory with name: no-name-c81c1788-d7b5-4191-95ee-9fb77d4a5160
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 32.4 K
2 | fc3  | Linear | 18.7 K
3 | fc4  | Linear | 24.7 K
--------------------------------
102 K     Trainable params
0         Non-trainable params
102 K     Total params
0.412     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 48.02it/s, v_num=3, train_loss=861.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 47.57it/s, v_num=3, train_loss=861.0]
[I 2023-05-04 15:05:06,782] Trial 0 finished with value: 2392.20751953125 and parameters: {'latent_dim': 63}. Best is trial 0 with value: 2392.20751953125.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 31.4 K
2 | fc3  | Linear | 18.2 K
3 | fc4  | Linear | 24.7 K
--------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.405     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 58.11it/s, v_num=4, train_loss=830.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 57.72it/s, v_num=4, train_loss=830.0]
[I 2023-05-04 15:05:23,613] Trial 1 finished with value: 2351.134521484375 and parameters: {'latent_dim': 61}. Best is trial 1 with value: 2351.134521484375.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 108 K 
2 | fc3  | Linear | 56.6 K
3 | fc4  | Linear | 24.7 K
--------------------------------
216 K     Trainable params
0         Non-trainable params
216 K     Total params
0.867     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 56.34it/s, v_num=5, train_loss=876.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 55.89it/s, v_num=5, train_loss=876.0]
[I 2023-05-04 15:05:41,434] Trial 2 finished with value: 2495.365234375 and parameters: {'latent_dim': 211}. Best is trial 1 with value: 2351.134521484375.

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 113 K 
2 | fc3  | Linear | 59.1 K
3 | fc4  | Linear | 24.7 K
--------------------------------
224 K     Trainable params
0         Non-trainable params
224 K     Total params
0.898     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 57.33it/s, v_num=6, train_loss=832.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 56.87it/s, v_num=6, train_loss=832.0]
[I 2023-05-04 15:05:57,861] Trial 3 finished with value: 2569.151123046875 and parameters: {'latent_dim': 221}. Best is trial 1 with value: 2351.134521484375.

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 15.9 K
2 | fc3  | Linear | 10.5 K
3 | fc4  | Linear | 24.7 K
--------------------------------
78.2 K    Trainable params
0         Non-trainable params
78.2 K    Total params
0.313     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 57.40it/s, v_num=7, train_loss=761.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 57.09it/s, v_num=7, train_loss=761.0]
[I 2023-05-04 15:06:15,607] Trial 4 finished with value: 2275.631591796875 and parameters: {'latent_dim': 31}. Best is trial 4 with value: 2275.631591796875.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 105 K 
2 | fc3  | Linear | 55.0 K
3 | fc4  | Linear | 24.7 K
--------------------------------
212 K     Trainable params
0         Non-trainable params
212 K     Total params
0.849     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 58.07it/s, v_num=8, train_loss=907.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 57.63it/s, v_num=8, train_loss=907.0]
[I 2023-05-04 15:06:32,468] Trial 5 finished with value: 2464.591796875 and parameters: {'latent_dim': 205}. Best is trial 4 with value: 2275.631591796875.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 24.7 K
2 | fc3  | Linear | 14.8 K
3 | fc4  | Linear | 24.7 K
--------------------------------
91.3 K    Trainable params
0         Non-trainable params
91.3 K    Total params
0.365     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 53.44it/s, v_num=9, train_loss=940.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 53.15it/s, v_num=9, train_loss=940.0]
[I 2023-05-04 15:06:51,359] Trial 6 finished with value: 2352.005615234375 and parameters: {'latent_dim': 48}. Best is trial 4 with value: 2275.631591796875.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 23.1 K
2 | fc3  | Linear | 14.1 K
3 | fc4  | Linear | 24.7 K
--------------------------------
89.0 K    Trainable params
0         Non-trainable params
89.0 K    Total params
0.356     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 57.74it/s, v_num=10, train_loss=941.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 57.40it/s, v_num=10, train_loss=941.0]
[I 2023-05-04 15:07:08,881] Trial 7 finished with value: 2336.918701171875 and parameters: {'latent_dim': 45}. Best is trial 4 with value: 2275.631591796875.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 28.3 K
2 | fc3  | Linear | 16.6 K
3 | fc4  | Linear | 24.7 K
--------------------------------
96.7 K    Trainable params
0         Non-trainable params
96.7 K    Total params
0.387     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 56.90it/s, v_num=11, train_loss=837.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 56.56it/s, v_num=11, train_loss=837.0]
[I 2023-05-04 15:07:27,452] Trial 8 finished with value: 2333.72314453125 and parameters: {'latent_dim': 55}. Best is trial 4 with value: 2275.631591796875.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 27.1 K
1 | fc2  | Linear | 115 K 
2 | fc3  | Linear | 59.9 K
3 | fc4  | Linear | 24.7 K
--------------------------------
226 K     Trainable params
0         Non-trainable params
226 K     Total params
0.907     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 54.92it/s, v_num=12, train_loss=954.0]  
`Trainer.fit` stopped: `max_epochs=10` reached.
Epoch 9: 100%|██████████| 91/91 [00:01<00:00, 54.46it/s, v_num=12, train_loss=954.0]
[I 2023-05-04 15:07:45,962] Trial 9 finished with value: 2482.10107421875 and parameters: {'latent_dim': 224}. Best is trial 4 with value: 2275.631591796875.
Best trial - Loss: 2275.6316
Best trial - Hyperparameters: {'latent_dim': 31}
```